In [2]:
# Ground modules
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
from Bio import SeqIO
from itertools import product
import random
from collections import Counter
import json
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

import logging
import subprocess
from multiprocessing.pool import ThreadPool
import joblib

# SCikitlearn modules :
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report , roc_auc_score

# Scipy modules : 
from scipy.stats import fisher_exact

In [3]:
# Make the blastp DB of all the dpo sequences :

path_seqbased = "/media/concha-eloko/Linux/PPT_clean/Seqbased_model"
path_db = f"{path_seqbased}/TropiSeq/TropiSeq_0.85.db"
path_work = "/media/concha-eloko/Linux/PPT_clean"


# Run makeblast command :
fasta_file = f"{path_seqbased}/cdhit_clusters_2912/0.85.out"

blast_command = f"makeblastdb -in {fasta_file} -dbtype prot -out {path_seqbased}/TropiSeq/TropiSeq_0.85.db"
#make_blast_process = subprocess.Popen(blast_command, shell =True, stdout = subprocess.PIPE, stderr=subprocess.STDOUT)
#mkblast_out, mkblast_err = make_blast_process.communicate()
#print(mkblast_out , mkblast_err)

# Relevant files :
dico_cluster = json.load(open(f"{path_seqbased}/dico_cluster.cdhit__0.85.json"))
dico_cluster_r = {ref_dpo : key_dpo for key_dpo,list_dpo in dico_cluster.items() for ref_dpo in list_dpo}

In [6]:
final_annotation = pd.read_csv(f"{path_work}/labeling_depo_clusters.PPT.0804.tsv", sep = "\t", header = 0)
final_annotation_tropiseq = final_annotation[final_annotation["Tropiseq_KLtypes"] != "None"]
final_annotation_tropiseq = final_annotation_tropiseq.drop_duplicates(subset = ["depo_cluster", "Tropiseq_KLtypes"])
final_annotation_tropiseq = final_annotation_tropiseq[["depo_cluster", "Tropiseq_KLtypes", "Tropiseq_scores"]]


In [5]:
dico_pred = json.load(open("/media/concha-eloko/Linux/PPT_clean/Seqbased_model/prediction_based.labeling.0604.json"))
dico_pred_correct_name = {f"Dpo_cdhit_{cluster.split('_')[1]}":hits  for cluster, hits in dico_pred.items()}
dico_pred_correct_name

{'Dpo_cdhit_0': {'KL103': 0.9887005649717514},
 'Dpo_cdhit_1': {},
 'Dpo_cdhit_2': {},
 'Dpo_cdhit_3': {},
 'Dpo_cdhit_4': {'KL54': 0.6317016672506052},
 'Dpo_cdhit_5': {},
 'Dpo_cdhit_6': {},
 'Dpo_cdhit_7': {},
 'Dpo_cdhit_8': {},
 'Dpo_cdhit_9': {},
 'Dpo_cdhit_10': {},
 'Dpo_cdhit_11': {},
 'Dpo_cdhit_12': {},
 'Dpo_cdhit_13': {},
 'Dpo_cdhit_14': {},
 'Dpo_cdhit_15': {'KL14': 0.9669649842305083},
 'Dpo_cdhit_16': {},
 'Dpo_cdhit_17': {},
 'Dpo_cdhit_18': {},
 'Dpo_cdhit_19': {},
 'Dpo_cdhit_20': {'KL1': 0.5049816914017605},
 'Dpo_cdhit_21': {},
 'Dpo_cdhit_22': {},
 'Dpo_cdhit_23': {},
 'Dpo_cdhit_24': {},
 'Dpo_cdhit_25': {'KL16': 0.6563300047113529},
 'Dpo_cdhit_26': {'KL164': 0.615},
 'Dpo_cdhit_27': {'KL46': 0.5539146456204256},
 'Dpo_cdhit_28': {},
 'Dpo_cdhit_29': {'KL57': 0.5119127698312795},
 'Dpo_cdhit_30': {},
 'Dpo_cdhit_31': {},
 'Dpo_cdhit_32': {'KL123': 0.5939826590887848},
 'Dpo_cdhit_33': {},
 'Dpo_cdhit_34': {},
 'Dpo_cdhit_35': {},
 'Dpo_cdhit_36': {},
 'Dpo_cdhi

***
## Make the predictions :

In [7]:
path_tmp =  "/media/concha-eloko/Linux/PPT_clean/Seqbased_model/tmp"
labels_blast=["qseqid", "sseqid", "pident", "length", "mismatch", "gapopen", "qstart", "qend", "sstart", "send", "evalue", "bitscore"]

def tmp_fasta_file(record , path_tmp) :
    name_file = "_".join(record.description.split(" "))
    path_fasta = f"{path_tmp}/{name_file}.fasta"
    length_seq = len(record.seq)
    with open(path_fasta, "w") as outfile :
        outfile.write(f">{record.description}\n{str(record.seq)}")
    return path_fasta , length_seq

def blast_seq(path_fasta, path_DB, path_tmp) :
    file_name = path_fasta.split("/")[-1]
    command = f"blastp -query {path_fasta} -db {path_DB} -out {path_tmp}/{file_name}.blast_out -outfmt 6 -evalue 1e-10"
    blastp_sub = subprocess.Popen(command ,shell = True, stdout = subprocess.PIPE, stderr = subprocess.STDOUT)
    out , err = blastp_sub.communicate()
    return f"{path_tmp}/{file_name}.blast_out"

def get_best_candidate(path_blast_out, length_seq, bitscore = 75) : 
    winner = 0
    labels_blast=["qseqid", "sseqid", "pident", "length", "mismatch", "gapopen", "qstart", "qend", "sstart", "send", "evalue", "bitscore"]
    blast_df = pd.read_csv(path_blast_out, sep = "\t", names = labels_blast)
    if len(blast_df) > 0 :
        row = blast_df.iloc[0] 
        if row["bitscore"] > bitscore and length_seq/int(row["length"])> 0.8:
            winner = dico_cluster_r[row["sseqid"]]
        else :
            winner = "No hits"
    else :
        winner = "No hits"
    return winner

def get_winner(record , path_tmp) :
    path_func , len_func = tmp_fasta_file(record, path_tmp)
    path_blast_out_func = blast_seq(path_func , path_db, path_tmp)
    winner = get_best_candidate(path_blast_out_func, len_func)
    return winner

> Other Bea : 

In [13]:
path_seq = "/media/concha-eloko/Linux/PPT_clean/in_vitro/Others_bea"

dico_seq = {record.description : record.seq for record in SeqIO.parse(f"{path_seq}/full_other_bea.multi.fasta", "fasta") if len(record.seq) >0}
set_records = [record for record in SeqIO.parse(f"{path_seq}/full_other_bea.multi.fasta", "fasta") if len(record.seq) > 0]


other_bea_winners = []
for record in tqdm(set_records) :
    winner = get_winner(record, path_tmp)
    if winner != "No hits" :
        hit = int(winner.split("_")[-1])
        results = dico_pred_correct_name.get(winner, {})
        a = (record.description.split(",")[0] , winner, results)
    else :
        results = "Null"
    a = (record.description.split(".")[0] , winner, results)
    other_bea_winners.append(a)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 18.75it/s]


In [14]:
other_bea_winners

[('AUV61507', 'Dpo_cdhit_56', {'KL64': 0.9037259998449553}),
 ('BAN78446', 'Dpo_cdhit_7', {}),
 ('AZF89844', 'Dpo_cdhit_120', {'KL24': 0.7575537340839898}),
 ('ALT58497', 'Dpo_cdhit_323', {'KL47': 0.9502903399584091}),
 ('ALT58498', 'Dpo_cdhit_124', {}),
 ('APZ82768', 'Dpo_cdhit_62', {'KL13': 0.5272369948063018}),
 ('QTP95996', 'Dpo_cdhit_37', {}),
 ('YP_003347555',
  'Dpo_cdhit_652',
  {'KL66': 0.689041095890411, 'KL74': 0.6920250246510266}),
 ('YP_003347556',
  'Dpo_cdhit_676',
  {'KL70': 0.8878475811944775, 'KL21': 0.6739516818124685})]

> Compile the results : 

In [15]:
TropiSeq_results = other_bea_winners
TropiSeq_results

[('AUV61507', 'Dpo_cdhit_56', {'KL64': 0.9037259998449553}),
 ('BAN78446', 'Dpo_cdhit_7', {}),
 ('AZF89844', 'Dpo_cdhit_120', {'KL24': 0.7575537340839898}),
 ('ALT58497', 'Dpo_cdhit_323', {'KL47': 0.9502903399584091}),
 ('ALT58498', 'Dpo_cdhit_124', {}),
 ('APZ82768', 'Dpo_cdhit_62', {'KL13': 0.5272369948063018}),
 ('QTP95996', 'Dpo_cdhit_37', {}),
 ('YP_003347555',
  'Dpo_cdhit_652',
  {'KL66': 0.689041095890411, 'KL74': 0.6920250246510266}),
 ('YP_003347556',
  'Dpo_cdhit_676',
  {'KL70': 0.8878475811944775, 'KL21': 0.6739516818124685})]

In [16]:
with open("/media/concha-eloko/Linux/PPT_clean/Seqbased_model.Other_Bea.results.bit75.2606.tsv" , "w") as outfile :
    for prot in TropiSeq_results :
        prot_name = prot[0].split("_A")[0]
        if prot[1] == "No hits" :
            outfile.write(f"{prot_name}\tNo_hits\n")
        elif prot[2] == {} :
            outfile.write(f"{prot_name}\tNo_associations\n")
        else :
            try :
                hits = [f"{kltype}:{round(score,3)}" for kltype, score in prot[2].items()]
                outfile.write(f"{prot_name}\t")
                outfile.write(" ; ".join(hits))
                outfile.write("\n")
            except Exception as e :
                print(prot, e)

***
### Predictions : 

In [17]:
predictions_Seqbased_df = pd.read_csv("/media/concha-eloko/Linux/PPT_clean/Seqbased_model.Other_Bea.results.bit75.2606.tsv", sep = "\t", names = ["protein", "predicitons"])
predictions_Seqbased_df

,protein,predicitons
0,AUV61507,KL64:0.904
1,BAN78446,No_associations
2,AZF89844,KL24:0.758
3,ALT58497,KL47:0.95
4,ALT58498,No_associations
5,APZ82768,KL13:0.527
6,QTP95996,No_associations
7,YP_003347555,KL66:0.689 ; KL74:0.692
8,YP_003347556,KL70:0.888 ; KL21:0.674
